# Visualizing Raw 2-Photon Images
Some Ophys NWB Files contain the raw imaging data from 2-Photon calcium imaging. The raw images, taken over time, make up a movie. Using mpl_interactions, we can view the data interactively. Since the raw data are very large, it's recommended that you stream it rather than download it.

### Environment Setup

In [ ]:
from dandi import dandiapi
import dandi
import fsspec
import pynwb
import h5py
from fsspec.implementations.cached import CachingFileSystem
from nwbwidgets import nwb2widget
from matplotlib import pyplot as plt
import numpy as np
from mpl_interactions import hyperslicer
import ipywidgets as widgets

### Streaming 2P File
Streaming a file from DANDI requires information about the file of interest. Set `dandiset_id` to be the ID of the dandiset you want, and set `filepath` to be the path of the file within the dandiset. The filepath can be found if you press on the `i` icon of a file and copy the `path` field that shows up in the resulting JSON. If you are accessing embargoed data, you will need to set `authenticate` to True, and `dandi_api_key` to your dandi api key.

In [ ]:
dandiset_id = "000336"
filepath = "1222982421/1222982421/1223100234_raw.nwb"
authenticate = True
dandi_api_key = "5c1eb544e6870c4c2ea676d22312cefb73e161e1"

In [ ]:
if authenticate:
    client = dandiapi.DandiAPIClient(token=dandi_api_key)
else:
    client = dandiapi.DandiAPIClient()
my_dandiset = client.get_dandiset(dandiset_id)

file = my_dandiset.get_asset_by_path(filepath)
base_url = file.client.session.head(file.base_download_url)
file_url = base_url.headers['Location']

print(f"Retrieved file url {file_url}")

In [ ]:
fs = CachingFileSystem(
    fs=fsspec.filesystem("http"),
    cache_storage="nwb-cache",  # Local folder for the cache
)

f = fs.open(file_url, "rb")
file = h5py.File(f)
io = pynwb.NWBHDF5IO(file=file, mode='r', load_namespaces=True)
nwb = io.read()

In [ ]:
io = pynwb.NWBHDF5IO("../../../data/dendrite_coupling/1223100234_raw.nwb", mode='r', load_namespaces=True)
nwb = io.read()

### TODO
We need to extract:
1. raw data
2. dff
3. corrected fluorescence
4. ROIs

tasks:
plot dff and fluorescence
plot raw over time
overlay ROIs onto raw slices

### Extracting Fluorescence

In [ ]:
dff = nwb.processing["ophys"]["dff"]
dff_trace = dff.roi_response_series["traces"].data
dff_timestamps = dff.roi_response_series["traces"].timestamps

flr = nwb.processing["ophys"]["corrected_fluorescence"]
flr_trace = flr.roi_response_series["traces"].data
flr_timestamps = flr.roi_response_series["traces"].timestamps

In [ ]:
dff_trace

In [ ]:
nwb.imaging_planes

In [ ]:
dff_trace.shape

### Extracting 2P Movie

In [ ]:
two_p = nwb.acquisition["raw_suite2p_motion_corrected"].data
print(two_p.shape[0])

### Extracting Regions of Interest

In [ ]:
roi_masks = nwb.processing["ophys"]["image_segmentation"].plane_segmentations["cell_specimen_table"].image_mask
roi_masks[:100]

In [ ]:
roi_masks = np.array(nwb.processing["ophys"]["images"]["segmentation_mask_image"])
print(roi_masks)

### Aligning 2P Movie with Fluorescence

In [ ]:
%matplotlib ipympl


### Viewing the Images
We can view the images with mpl_interactions.hyperslicer. The number of frames is printed in the first cell below. Because the image data are so large, it is advised the you select a subsection of these images to view. Set `start_frame` to the first image you want to be displayed, and `end_frame` to be the last. The larger this interval, the longer it will take to display. It is recommended the interval is less than 2000.

In [ ]:
start_frame = 0
end_frame = start_frame + 200
two_p_subset = np.array(two_p[start_frame:end_frame])

In [ ]:
%matplotlib

print(dff_timestamps.shape)
print(dff_trace.shape)

fig, ax = plt.subplots(figsize=(8,6))
ax.plot(dff_timestamps[start_frame: end_frame], dff_trace[start_frame: end_frame])

plt.show()

In [ ]:
roi_masks_3d = np.reshape(roi_masks, (1,) + roi_masks.shape)
m = two_p_subset.mean()
roi_masks_3d = np.multiply(roi_masks_3d, m*10)

two_p_masked = np.add(two_p_subset, roi_masks_3d)

In [ ]:
print(two_p_subset.shape)
print(roi_masks.shape)
print(two_p_masked.shape)

In [ ]:
%matplotlib ipympl
plt.imshow(roi_masks)

In [ ]:
%matplotlib ipympl
# iplt.imshow(data, interpolation='nearest')
fig, ax = plt.subplots()
#plt.imshow(roi_masks)
view = hyperslicer(two_p_masked,play_buttons=True)